In [1]:
from gbp.gbp import *
from gbp.factor import *
from gbp.grid import *

Jax plugin configuration error: Plugin module %s could not be loaded
Traceback (most recent call last):
  File "/home/yuzhou/anaconda3/envs/jax-gpu/lib/python3.11/site-packages/jax/_src/xla_bridge.py", line 420, in discover_pjrt_plugins
    plugin_module = importlib.import_module(plugin_module_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/yuzhou/anaconda3/envs/jax-gpu/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", li

In [2]:
positions, prior_meas, between_meas = generate_grid_slam_data(H=16, W=16, 
                                    prior_noise_std=1, odom_noise_std=0.1, seed=34)
varis, prior_facs, between_facs = build_pose_slam_graph(N=256, prior_meas=prior_meas, between_meas=between_meas, 
                                                        prior_std=1, odom_std=0.01,
                                                        Ni_v=10, D=2)


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
varis, prior_facs, between_facs, energy_log, linpoints_log = gbp_solve(
    varis, prior_facs, between_facs, num_iters=30, visualize=True
)


In [4]:
# build the coarse-level pose SLAM grid
varis_coarse, prior_facs_coarse, horizontal_facs_coarse, vertical_facs_coarse  = build_coarse_slam_graph(
    varis_fine=varis,
    prior_facs_fine=prior_facs,
    between_facs_fine=between_facs,
    H=16, W=16,
    stride = 2,
    prior_std = 1.0,
    between_std = 0.1,
)



In [5]:
# Solve the coarse-level grid with GBP
varis_coarse, prior_facs_coarse, horizontal_facs_coarse, vertical_facs_coarse, energy_log_coarse, linpoints_log_coarse = gbp_solve_coarse(
    varis_coarse, prior_facs_coarse, horizontal_facs_coarse, vertical_facs_coarse, 
    num_iters=30, visualize=True
)

In [6]:
varis_coarser, prior_facs_coarser, horizontal_facs_coarser, vertical_facs_coarser = build_coarser_slam_graph(
    varis_coarse=varis_coarse,
    prior_facs_coarse=prior_facs_coarse,
    horizontal_between_facs=horizontal_facs_coarse,
    vertical_between_facs=vertical_facs_coarse,
    H=16, W=16,
    stride=2
)


In [7]:
# Solve the coarser-level grid with GBP
varis_coarser, prior_facs_coarser, horizontal_facs_coarser, vertical_facs_coarser, energy_log_coarser, linpoints_log_coarser = gbp_solve_coarse(
    varis_coarser, prior_facs_coarser, horizontal_facs_coarser, vertical_facs_coarser, 
    num_iters=30, visualize=True, prior_h=h6_fn, between_h=[h7_fn,h8_fn]
)

In [8]:
print(energy_log)
print(energy_log_coarse)
print(energy_log_coarser)

[ 33161.6    102010.07    22780.06    15203.153   12360.459   10915.94
  10052.285    9425.442    8935.662    8518.978    8152.6465   7817.998
   7510.232    7223.692    6957.0493   6707.4355   6473.83     6254.6143
   6048.8677   5855.652    5673.994    5503.3296   5342.692    5191.752
   5049.5684   4915.96     4790.03     4671.6924   4560.1084   4455.25  ]
[33161.6    34650.535  21506.887  15787.97   10493.718   7174.5854
  5748.1846  5095.4736  4629.01    4272.0684  3985.311   3759.7793
  3576.5498  3431.256   3311.9421  3216.838   3138.7188  3076.296
  3024.8203  2983.5063  2949.3164  2921.7202  2898.7527  2880.078
  2864.4268  2851.5757  2840.7065  2831.6738  2823.9485  2817.4346]
[33161.6    40991.945  23785.008   7921.0674  3369.541   3178.7605
  2887.9172  2766.1409  2744.7656  2744.6123  2742.2085  2742.4343
  2741.869   2741.9287  2741.7512  2741.7644  2741.703   2741.6992
  2741.673   2741.6665  2741.6538  2741.6477  2741.6401  2741.6353
  2741.6306  2741.6272  2741.6238  2

In [9]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


# Visibility control for toggle buttons
def make_visibility(show_fine, show_coarse, show_coarser):
    return [show_fine, show_coarse, show_coarser]


# Automatically compute axis ranges and enforce equal scaling
all_points = np.concatenate([
    linpoints_log.reshape(-1, 2),
    linpoints_log_coarse.reshape(-1, 2),
    linpoints_log_coarser.reshape(-1, 2)
], axis=0)

x_min, y_min = all_points.min(axis=0)
x_max, y_max = all_points.max(axis=0)

x_range = x_max - x_min
y_range = y_max - y_min
max_range = max(x_range, y_range) / 2

x_center = (x_max + x_min) / 2
y_center = (y_max + y_min) / 2

x_min_equal = x_center - max_range - 0.001 * max_range
x_max_equal = x_center + max_range + 0.001 * max_range
y_min_equal = y_center - max_range - 0.001 * max_range
y_max_equal = y_center + max_range + 0.001 * max_range

# Get dimensions for each level
num_iters = linpoints_log.shape[0]
fine_points = linpoints_log.shape[1]
coarse_points = linpoints_log_coarse.shape[1]
coarser_points = linpoints_log_coarser.shape[1]

# Color assignment for each level
fine_base = px.colors.qualitative.Set3
fine_colors = (fine_base * ((fine_points // len(fine_base)) + 1))[:fine_points]
coarse_colors = px.colors.qualitative.Plotly * 10
coarser_colors = px.colors.qualitative.D3 * 10

# Precompute color lists for coarse and coarser points
coarse_color_list = [coarse_colors[i] for i in range(coarse_points) for _ in range(4)]
coarser_color_list = [coarser_colors[i] for i in range(coarser_points) for _ in range(16)]

# Create animation frames (optimized version)
frames = []
for t in range(num_iters):
    fine_trace = go.Scatter(
        x=linpoints_log[t, :, 0],
        y=linpoints_log[t, :, 1],
        mode='markers',
        marker=dict(size=4, color=fine_colors),
        name='fine',
        showlegend=False
    )

    coarse_pts = linpoints_log_coarse[t].reshape(-1, 2)
    coarse_trace = go.Scatter(
        x=coarse_pts[:, 0],
        y=coarse_pts[:, 1],
        mode='markers',
        marker=dict(size=4, color=coarse_color_list),
        name='coarse',
        showlegend=False
    )

    coarser_pts = linpoints_log_coarser[t].reshape(-1, 2)
    coarser_trace = go.Scatter(
        x=coarser_pts[:, 0],
        y=coarser_pts[:, 1],
        mode='markers',
        marker=dict(size=4, color=coarser_color_list),
        name='coarser',
        showlegend=False
    )

    # Energy plots (growing line per iteration)
    energy_trace_fine = go.Scatter(
        x=np.arange(t+1),
        y=energy_log[:t+1],
        mode='lines+markers',
        line=dict(color='pink'),
        marker=dict(size=3),
        name='Energy Fine',
        xaxis='x2',
        yaxis='y2',
        showlegend=False
    )

    energy_trace_coarse = go.Scatter(
        x=np.arange(t+1),
        y=energy_log_coarse[:t+1],
        mode='lines+markers',
        line=dict(color='lightblue'),
        marker=dict(size=3),
        name='Energy Coarse',
        xaxis='x2',
        yaxis='y2',
        showlegend=False
    )

    energy_trace_coarser = go.Scatter(
        x=np.arange(t+1),
        y=energy_log_coarser[:t+1],
        mode='lines+markers',
        line=dict(color='lightgreen'),
        marker=dict(size=3),
        name='Energy Coarser',
        xaxis='x2',
        yaxis='y2',
        showlegend=False
    )

    frames.append(go.Frame(
        name=str(t),
        data=[fine_trace, coarse_trace, coarser_trace,
                energy_trace_fine, energy_trace_coarse, energy_trace_coarser]
    ))

# Define layout and animation controls
layout = go.Layout(
    width=1000,
    height=650,
    title=dict(
        text='Multi-Scale GBP Animation',
        x=0.5,
        xanchor='center',
        y=0.95,
        yanchor='top',
        font=dict(
            family='Times New Roman, serif',
            size=20,
            color='black'
        )
    ),
    margin=dict(l=0, r=0, t=0.2*650, b=0.1*650),

    xaxis1=dict(                    # Position plot x-axis (position)
        range=[x_min_equal, x_max_equal], 
        domain=[0.4, 1], 
        constrain='domain', 
        anchor='y1'),
    yaxis1=dict(                # Position plot y-axis (position)
        range=[y_min_equal, y_max_equal], 
        domain=[0, 1], 
        scaleanchor='x1', 
        scaleratio=1, 
        anchor='x1'),


    xaxis2=dict(                     # Energy plot x-axis (iteration)
        domain=[0.0, 0.35],
        anchor="y2",
        title="Iteration",
    ),
    yaxis2=dict(                     # Energy plot y-axis (energy)
        domain=[0.0, 0.45],
        anchor="x2",
        title="Energy",
    ),


    updatemenus=[
        {
            "type": "buttons",
            "direction": "down",
            "x": 0.44,
            "xanchor": "right",
            "y": 0.6,
            "yanchor": "top",
            "pad": {"r": 0, "t": 0},
            "buttons": [
                {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}]},
                {"label": "Pause", "method": "animate", "args": [[None], {"mode": "immediate", "frame": {"duration": 0, "redraw": False}, "transition": {"duration": 0}}]},
                {"label": "Stop", "method": "animate", "args": [["0"], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}}]},
            ]
        },
        {
            "type": "buttons",
            "direction": "right",
            "x": 0.6,
            "xanchor": "left",
            "y": 1.1,
            "yanchor": "top",
            "pad": {"r": 0, "t": 0},
            "buttons": [
                {"label": "Fine", "method": "update", "args": [{"visible": make_visibility(True, False, False)}]},
                {"label": "Coarse", "method": "update", "args": [{"visible": make_visibility(False, True, False)}]},
                {"label": "Coarser", "method": "update", "args": [{"visible": make_visibility(False, False, True)}]},
            ]
        }
    ]
)




# Create and show the figure
fig = go.Figure(data=frames[0].data, 
                frames=frames, 
                layout=layout)
fig.show()




In [10]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Function to ensure only point visibility is toggled by buttons,
# while energy curves always stay visible (last 3 traces)
def make_visibility(show_fine, show_coarse, show_coarser):
    return [show_fine, show_coarse, show_coarser, True, True, True]

# Compute overall axis bounds for equal scaling
all_points = np.concatenate([
    linpoints_log.reshape(-1, 2),
    linpoints_log_coarse.reshape(-1, 2),
    linpoints_log_coarser.reshape(-1, 2)
], axis=0)

x_min, y_min = all_points.min(axis=0)
x_max, y_max = all_points.max(axis=0)

x_range = x_max - x_min
y_range = y_max - y_min
max_range = max(x_range, y_range) / 2

x_center = (x_max + x_min) / 2
y_center = (y_max + y_min) / 2

x_min_equal = x_center - max_range - 0.001 * max_range
x_max_equal = x_center + max_range + 0.001 * max_range
y_min_equal = y_center - max_range - 0.001 * max_range
y_max_equal = y_center + max_range + 0.001 * max_range

# Dimensions
num_iters = linpoints_log.shape[0]
fine_points = linpoints_log.shape[1]
coarse_points = linpoints_log_coarse.shape[1]
coarser_points = linpoints_log_coarser.shape[1]

# Assign colors to each point group
fine_base = px.colors.qualitative.Set3
fine_colors = (fine_base * ((fine_points // len(fine_base)) + 1))[:fine_points]
coarse_colors = px.colors.qualitative.Plotly * 10
coarser_colors = px.colors.qualitative.D3 * 10

# Precompute color assignments per point
coarse_color_list = [coarse_colors[i] for i in range(coarse_points) for _ in range(4)]
coarser_color_list = [coarser_colors[i] for i in range(coarser_points) for _ in range(16)]

# Create animation frames
frames = []
for t in range(num_iters):
    # Fine-level points (right side)
    fine_trace = go.Scatter(
        x=linpoints_log[t, :, 0],
        y=linpoints_log[t, :, 1],
        mode='markers',
        marker=dict(size=4, color=fine_colors),
        name='fine',
        showlegend=False,
        xaxis='x1',
        yaxis='y1'
    )

    # Coarse-level points (right side)
    coarse_pts = linpoints_log_coarse[t].reshape(-1, 2)
    coarse_trace = go.Scatter(
        x=coarse_pts[:, 0],
        y=coarse_pts[:, 1],
        mode='markers',
        marker=dict(size=4, color=coarse_color_list),
        name='coarse',
        showlegend=False,
        xaxis='x1',
        yaxis='y1'
    )

    # Coarser-level points (right side)
    coarser_pts = linpoints_log_coarser[t].reshape(-1, 2)
    coarser_trace = go.Scatter(
        x=coarser_pts[:, 0],
        y=coarser_pts[:, 1],
        mode='markers',
        marker=dict(size=4, color=coarser_color_list),
        name='coarser',
        showlegend=False,
        xaxis='x1',
        yaxis='y1'
    )

    # Sliding window for energy plot (left bottom)
    start_idx = max(0, t - 4)
    x_vals = np.arange(start_idx, t + 1)

    # Energy curves: fine, coarse, coarser
    energy_trace_fine = go.Scatter(
        x=x_vals,
        y=energy_log[start_idx:t + 1],
        mode='lines+markers',
        line=dict(color='pink'),
        marker=dict(size=3),
        name='Energy Fine',
        xaxis='x2',
        yaxis='y2',
        showlegend=False
    )

    energy_trace_coarse = go.Scatter(
        x=x_vals,
        y=energy_log_coarse[start_idx:t + 1],
        mode='lines+markers',
        line=dict(color='lightblue'),
        marker=dict(size=3),
        name='Energy Coarse',
        xaxis='x2',
        yaxis='y2',
        showlegend=False
    )

    energy_trace_coarser = go.Scatter(
        x=x_vals,
        y=energy_log_coarser[start_idx:t + 1],
        mode='lines+markers',
        line=dict(color='lightgreen'),
        marker=dict(size=3),
        name='Energy Coarser',
        xaxis='x2',
        yaxis='y2',
        showlegend=False
    )

    # Dynamic y-range for zoomed-in energy chart
    dynamic_ymax = max(
        energy_log[start_idx:t + 1].max(),
        energy_log_coarse[start_idx:t + 1].max(),
        energy_log_coarser[start_idx:t + 1].max()
    )
    dynamic_ymin = min(
        energy_log[start_idx:t + 1].min(),
        energy_log_coarse[start_idx:t + 1].min(),
        energy_log_coarser[start_idx:t + 1].min()
    )


    # Create and append this frame
    frames.append(go.Frame(
        name=str(t),
        data=[
            fine_trace, coarse_trace, coarser_trace,
            energy_trace_fine, energy_trace_coarse, energy_trace_coarser
        ],
        layout=go.Layout(
            xaxis2=dict(range=[start_idx, t], domain=[0.1, 0.35], anchor='y2'),
            yaxis2=dict(range=[0, dynamic_ymax], domain=[0, 0.45], anchor='x2')
        )
    ))

# Static layout (position and energy plots, animation buttons)
layout = go.Layout(
    width=1000,
    height=650,
    title=dict(
        text='Multi-Scale GBP Animation',
        x=0.5,
        xanchor='center',
        y=0.95,
        yanchor='top',
        font=dict(family='Times New Roman, serif', size=20, color='black')
    ),
    margin=dict(l=0, r=0, t=0.2 * 650, b=0.1 * 650),


    annotations=[
        dict(
            x=0.98, y=0.95,
            xref='x2 domain', yref='y2 domain',
            text="<span style='color:pink; font-size:10px;'>Fine</span><br>"
                "<span style='color:lightblue; font-size:10px;'>Coarse</span><br>"
                "<span style='color:lightgreen; font-size:10px;'>Coarser</span>",
            showarrow=False,
            align='right',
            bgcolor='rgba(255,255,255,0.6)',
            bordercolor='lightgray',
            borderwidth=0.5,
            borderpad=2,
        )
    ],


    # Main position plot (right)
    xaxis1=dict(range=[x_min_equal, x_max_equal], domain=[0.4, 1], constrain='domain', anchor='y1'),
    yaxis1=dict(range=[y_min_equal, y_max_equal], domain=[0, 1], scaleanchor='x1', scaleratio=1, anchor='x1'),

    # Energy plot (left-bottom)
    xaxis2=dict(domain=[0.1, 0.35], anchor="y2", title="Iteration", automargin=False),
    yaxis2=dict(domain=[0, 0.45], anchor="x2", title="Energy", automargin=False),

    # Button menus
    updatemenus=[
        {
            "type": "buttons",
            "direction": "down",
            "x": 0.44,
            "xanchor": "right",
            "y": 0.6,
            "yanchor": "top",
            "pad": {"r": 0, "t": 0},
            "buttons": [
                {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}]},
                {"label": "Pause", "method": "animate", "args": [[None], {"mode": "immediate", "frame": {"duration": 0, "redraw": False}, "transition": {"duration": 0}}]},
                {"label": "Stop", "method": "animate", "args": [["0"], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}}]}
            ]
        },
        {
            "type": "buttons",
            "direction": "right",
            "x": 0.6,
            "xanchor": "left",
            "y": 1.1,
            "yanchor": "top",
            "pad": {"r": 0, "t": 0},
            "buttons": [
                {"label": "Fine", "method": "update", "args": [{"visible": make_visibility(True, False, False)}]},
                {"label": "Coarse", "method": "update", "args": [{"visible": make_visibility(False, True, False)}]},
                {"label": "Coarser", "method": "update", "args": [{"visible": make_visibility(False, False, True)}]}
            ]
        }
    ]


    
)

# Create and display figure
fig = go.Figure(data=frames[0].data, frames=frames, layout=layout)
fig.show()


In [62]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Dummy placeholders for demonstration (you should replace these with real data)
num_iters = 10
fine_points = 5
coarse_points = 3
coarser_points = 2

linpoints_log = np.random.rand(num_iters, fine_points, 2)
linpoints_log_coarse = np.random.rand(num_iters, coarse_points, 2)
linpoints_log_coarser = np.random.rand(num_iters, coarser_points, 2)
energy_log = np.linspace(1, 0.1, num_iters)
energy_log_coarse = np.linspace(1.2, 0.2, num_iters)
energy_log_coarser = np.linspace(1.5, 0.3, num_iters)

# Function to control visibility of point groups, energy curves always visible
def make_visibility(show_fine, show_coarse, show_coarser):
    return [show_fine, show_coarse, show_coarser, True, True, True]

# Compute equal aspect ratio bounds
all_points = np.concatenate([
    linpoints_log.reshape(-1, 2),
    linpoints_log_coarse.reshape(-1, 2),
    linpoints_log_coarser.reshape(-1, 2)
], axis=0)
x_min, y_min = all_points.min(axis=0)
x_max, y_max = all_points.max(axis=0)
x_center = (x_max + x_min) / 2
y_center = (y_max + y_min) / 2
max_range = max(x_max - x_min, y_max - y_min) / 2
x_min_equal = x_center - max_range
x_max_equal = x_center + max_range
y_min_equal = y_center - max_range
y_max_equal = y_center + max_range

# Colors
fine_colors = px.colors.qualitative.Set3 * ((fine_points // 12) + 1)
coarse_colors = px.colors.qualitative.Plotly * ((coarse_points // 10) + 1)
coarser_colors = px.colors.qualitative.D3 * ((coarser_points // 10) + 1)

coarse_color_list = [coarse_colors[i] for i in range(coarse_points) for _ in range(4)]
coarser_color_list = [coarser_colors[i] for i in range(coarser_points) for _ in range(16)]

# Frames
frames = []
for t in range(num_iters):
    start_idx = max(0, t - 4)
    x_vals = np.arange(start_idx, t + 1)

    traces = [
        go.Scatter(x=linpoints_log[t, :, 0], y=linpoints_log[t, :, 1], mode='markers',
                   marker=dict(size=4, color=fine_colors[:fine_points]), name='fine',
                   showlegend=False, xaxis='x1', yaxis='y1'),

        go.Scatter(x=linpoints_log_coarse[t, :, 0], y=linpoints_log_coarse[t, :, 1], mode='markers',
                   marker=dict(size=4, color=coarse_color_list[:coarse_points]), name='coarse',
                   showlegend=False, xaxis='x1', yaxis='y1'),

        go.Scatter(x=linpoints_log_coarser[t, :, 0], y=linpoints_log_coarser[t, :, 1], mode='markers',
                   marker=dict(size=4, color=coarser_color_list[:coarser_points]), name='coarser',
                   showlegend=False, xaxis='x1', yaxis='y1'),

        go.Scatter(x=x_vals, y=energy_log[start_idx:t+1], mode='lines+markers',
                   line=dict(color='pink'), marker=dict(size=3), name='Energy Fine',
                   xaxis='x2', yaxis='y2', showlegend=False),

        go.Scatter(x=x_vals, y=energy_log_coarse[start_idx:t+1], mode='lines+markers',
                   line=dict(color='lightblue'), marker=dict(size=3), name='Energy Coarse',
                   xaxis='x2', yaxis='y2', showlegend=False),

        go.Scatter(x=x_vals, y=energy_log_coarser[start_idx:t+1], mode='lines+markers',
                   line=dict(color='lightgreen'), marker=dict(size=3), name='Energy Coarser',
                   xaxis='x2', yaxis='y2', showlegend=False)
    ]

    annotations = [
        dict(text="Fine", x=0.35, y=0.45, xref='paper', yref='paper',
             showarrow=False, font=dict(size=9, color='pink'), xanchor='right'),
        dict(text="Coarse", x=0.35, y=0.42, xref='paper', yref='paper',
             showarrow=False, font=dict(size=9, color='lightblue'), xanchor='right'),
        dict(text="Coarser", x=0.35, y=0.39, xref='paper', yref='paper',
             showarrow=False, font=dict(size=9, color='lightgreen'), xanchor='right')
    ]

    frames.append(go.Frame(name=str(t), data=traces,
                           layout=go.Layout(
                               xaxis2=dict(range=[start_idx, t+1]),
                               yaxis2=dict(range=[0, 1.6]),
                               annotations=annotations
                           )))

# Layout
layout = go.Layout(
    width=1000,
    height=650,
    title=dict(text='Multi-Scale GBP Animation', x=0.5, xanchor='center', y=0.95),
    margin=dict(l=0, r=0, t=130, b=65),

    xaxis1=dict(range=[x_min_equal, x_max_equal], domain=[0.4, 1], anchor='y1'),
    yaxis1=dict(range=[y_min_equal, y_max_equal], domain=[0, 1], scaleanchor='x1', scaleratio=1, anchor='x1'),

    xaxis2=dict(domain=[0.1, 0.35], anchor="y2", title="Iteration"),
    yaxis2=dict(domain=[0.05, 0.45], anchor="x2", title="Energy"),

    updatemenus=[
        dict(type="buttons", direction="down", x=0.44, xanchor="right", y=0.6, yanchor="top",
             buttons=[
                 dict(label="Play", method="animate", args=[None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}]),
                 dict(label="Pause", method="animate", args=[[None], {"mode": "immediate", "frame": {"duration": 0}, "transition": {"duration": 0}}]),
                 dict(label="Stop", method="animate", args=[["0"], {"mode": "immediate", "frame": {"duration": 0}, "transition": {"duration": 0}}])
             ]),
        dict(type="buttons", direction="right", x=0.6, xanchor="left", y=1.1, yanchor="top",
             buttons=[
                 dict(label="Fine", method="update", args=[{"visible": make_visibility(True, False, False)}]),
                 dict(label="Coarse", method="update", args=[{"visible": make_visibility(False, True, False)}]),
                 dict(label="Coarser", method="update", args=[{"visible": make_visibility(False, False, True)}])
             ])
    ]
)

# Create and show figure
fig = go.Figure(data=frames[0].data, layout=layout, frames=frames)
fig.show()
